In [1]:
%load_ext jupyter_black

## Load Packages <a class="anchor" id="load_packages"></a>

Packages relavant to th code are loaded below. Additionally, specific options or parameters are also set. For example, display options are set for `pandas` and the environment path to the environment variable file is provided for the `dotenv` package.

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import sys
import pandas as pd
import numpy as np
import requests
from spotify_dl import spotify_dl
from pathlib import Path
import time
import os
from dotenv import load_dotenv  # changed magic command to explicit load
import librosa
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import pairwise
from sklearn.model_selection import train_test_split
from typing import List
from flask import Flask, redirect, request
import random
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from skimage.transform import resize


pd.set_option("display.max_rows", None)  # pandas dataframe formatting options
pd.set_option("display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format


custom_env_path = "../../brainstation_capstone_cfg.env"  # environment variables file

## Setting Environment Variables <a class="anchor" id="set_env_vars"></a>

In [3]:
# Spotify Developer Credentials
load_dotenv(dotenv_path=custom_env_path)
CLIENT_ID = os.environ.get("SPOTIPY_CLIENT_ID")
# client ID from app
CLIENT_SECRET = os.environ.get("SPOTIPY_CLIENT_SECRET")
# client secret from app
REDIRECT_URI = os.environ.get("REDIRECT_URI")
# redirect URI - the URI used here matches the one used within the app
SCOPE = "{} {}".format(os.environ.get("SCOPE_PUBLIC"), os.environ.get("SCOPE_PRIVATE"))
# formatted the scope this way to allow for custom configurations in the future
USERNAME = os.environ.get("SPOTIFY_USERNAME")
# Spotify username

## Feature Function Definitions

In [4]:
# How was this solved? sampling rate must be explicitly passed to every function
# This includes the display function itself!
# Hence, the sampling rate is now an output from the function itself
def get_mfcc(y, sr):
    mfcc = librosa.power_to_db(librosa.feature.mfcc(y=y, sr=sr), ref=np.max)
    return mfcc


def get_melspectrogram(y, sr):
    melspectrogram = librosa.power_to_db(
        librosa.feature.melspectrogram(y=y, sr=sr), ref=np.max
    )
    # this is a power spectrum (amplitude squared)
    return melspectrogram


def get_chroma_vector(y, sr):
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    return chroma


def get_tonnetz(y, sr):
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr)
    return tonnetz


def get_feature(input_file_path, track_id):
    # Load data
    y, sr = librosa.load(
        input_file_path,
        sr=None,
        offset=10,
        duration=120,
    )
    # Extracting MFCC feature
    mfcc = get_mfcc(y, sr)
    # Move save step to this point!
    np.save(f"../data/vectorized_mp3s/raw/mfcc_{track_id}.npy", mfcc)
    mfcc_mean = mfcc.mean(axis=1)
    mfcc_min = mfcc.min(axis=1)
    mfcc_max = mfcc.max(axis=1)
    mfcc_feature = np.concatenate((mfcc_mean, mfcc_min, mfcc_max))

    # Need to move the save to before the reduction of features

    # Extracting Mel Spectrogram feature
    melspectrogram = get_melspectrogram(y, sr)
    np.save(
        f"../data/vectorized_mp3s/raw/melspectrogram_{track_id}.npy",
        melspectrogram,
    )
    melspectrogram_mean = melspectrogram.mean(axis=1)
    melspectrogram_min = melspectrogram.min(axis=1)
    melspectrogram_max = melspectrogram.max(axis=1)
    melspectrogram_feature = np.concatenate(
        (melspectrogram_mean, melspectrogram_min, melspectrogram_max)
    )

    # Extracting chroma vector feature
    chroma = get_chroma_vector(y, sr)
    np.save(f"../data/vectorized_mp3s/raw/chroma_{track_id}.npy", chroma)
    chroma_mean = chroma.mean(axis=1)
    chroma_min = chroma.min(axis=1)
    chroma_max = chroma.max(axis=1)
    chroma_feature = np.concatenate((chroma_mean, chroma_min, chroma_max))

    # Extracting tonnetz feature
    tntz = get_tonnetz(y, sr)
    np.save(f"../data/vectorized_mp3s/raw/tonnetz_{track_id}.npy", tntz)
    tntz_mean = tntz.mean(axis=1)
    tntz_min = tntz.min(axis=1)
    tntz_max = tntz.max(axis=1)
    tntz_feature = np.concatenate((tntz_mean, tntz_min, tntz_max))

    # return chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature
    # this returns both the pairwise vector feature and the CNN feature
    pairwise_feature = np.concatenate(
        (chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature)
    )
    height = 224
    width = 224
    channels = 3
    # for file_path in file_paths:
    #     if "1ZB2qWsheGabSEYvBYxjKn" in str(file_path):
    #         melspectrogram = np.load(
    #             f"../data/vectorized_mp3s/raw/melspectrogram_{track_id}.npy"
    #         )
    #         chroma = np.load(f"../data/vectorized_mp3s/raw/chroma_{track_id}.npy")
    #         tonnetz = np.load(f"../data/vectorized_mp3s/raw/tonnetz_{track_id}.npy")
    mel_resize = resize(melspectrogram, (height, width))
    chroma_resize = resize(chroma, (height, width))
    tntz_resize = resize(tntz, (height, width))
    cnn_feature = np.concatenate((chroma_resize, mel_resize, tntz_resize))
    return pairwise_feature, cnn_feature


# May not have use of this function yet....need to fix pairwise comparison
# Save spectra one stage before as separate vectors BEFORE concatenation
# def create_spectra(spec_data, sr, type, track_id):
#     if type == "mfcc":
#         plt.figure()
#         librosa.display.specshow(mel_data, y_axis="mel", x_axis="time", sr=sr)
#         plt.title("Mel Frequency Cepstral Coefficients")
#         plt.colorbar(format="%+2.0f dB")
#         plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
#     if type == "melspectrogram":
#         plt.figure()
#         librosa.display.specshow(mel_data, y_axis="mel", x_axis="time", sr=sr)
#         plt.title("Melspectrogram")
#         plt.colorbar(format="%+2.0f dB")
#         plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
#     if type == "chroma":
#         plt.figure()
#         librosa.display.specshow(
#             mel_data, y_axis="chroma", x_axis="time", sr=sr, vmin=0, vmax=1
#         )
#         plt.title("Chroma STFT")
#         plt.colorbar(format="%+2.0f dB")
#         plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
#     if type == "tonnetz":
#         plt.figure()
#         librosa.display.specshow(
#             mel_data, y_axis="tonnetz", x_axis="time", sr=sr, vmin=0, vmax=1
#         )
#         plt.title("Tonnetz")
#         plt.colorbar(format="%+2.0f dB")
#         plt.savefig(f"../data/mp3_spectra/{track_id}.png", bbox_inches="tight")
# Will have to get the track_id to save the file
# Not sure if the spectra must have the axis labels or the title

In [5]:
# test_file = "/Users/vii/repos/brainstation_capstone/data/mp3s/1ZB2qWsheGabSEYvBYxjKn/Take on Me/Weezer - Take on Me.mp3"
# track_id = "1ZB2qWsheGabSEYvBYxjKn"
# chroma_feature, melspectrogram_feature, mfcc_feature, tntz_feature = get_feature(
#     test_file, track_id
# )

In [6]:
# chroma_feature

In [7]:
# chroma_feature.shape

In [8]:
# chroma_npy = np.load("../data/vectorized_mp3s/chroma_1ZB2qWsheGabSEYvBYxjKn.npy")
# chroma_npy

In [9]:
# x = np.array([1, 2])

In [10]:
# np.array_equal(
#     chroma_feature, chroma_npy
# )  # test if the two arrays are the same after saving and loading

In [11]:
# np.array_equal(chroma_npy, x)  # sanity check

## Vectorize Data

In [12]:
data_dir = Path("../data/mp3s/")
output_dir = Path("../data/vectorized_mp3s/pairwise_parquets/")
path_glob = data_dir.rglob("*.mp3")
file_paths = []
for file_path in path_glob:
    file_paths.append(file_path)  # creates a list for repeated iteration
    # if this is not done, the .rglob command above has to be repeated to regenerate iterator
len(file_paths)  # number of mp3s in directory

11578

In [13]:
data_dir = Path("../data/mp3s/")
output_dir = Path("../data/vectorized_mp3s/pairwise_parquets/")
downloaded_path_glob = data_dir.rglob("*.mp3")
output_path_glob = output_dir.rglob("*.parquet")
count = 1
file_paths = [file_path for file_path in downloaded_path_glob]
print("Number of MP3 Files: ", len(file_paths), "\n")
vectorized_track_ids = [file_path.stem for file_path in output_path_glob]
for file_path in file_paths:
    print(f"{count}. FILE PATH: \n", f"{file_path}")
    path_split = str(file_path).split("/")
    track_id = path_split[3]
    if (len(vectorized_track_ids) > 0) & (track_id in vectorized_track_ids):
        print(f"{track_id} has already been vectorized...skipping...")
        count += 1
    else:
        pairwise_track, cnn_track = get_feature(file_path, track_id)
        vectorized_df = pd.DataFrame(pairwise_track).T
        vectorized_df["track_id"] = track_id
        vectorized_df = vectorized_df.set_index(vectorized_df.track_id).drop(
            columns="track_id"
        )
        vectorized_df.columns = vectorized_df.columns.astype(str)
        vectorized_df.to_parquet(
            f"../data/vectorized_mp3s/pairwise_parquets/{track_id}.parquet"
        )
        np.save(f"../data/mp3_spectra/cnn_npys/{track_id}.npy", cnn_track)
        count += 1

5817. FILE PATH: 
 ../data/mp3s/0RBctV92Wp5NSYUKVeTHha/Swelling of Pride/Steve Byrne - Swelling of Pride.mp3
5818. FILE PATH: 
 ../data/mp3s/1ghqlcFUP85h1Gxn4Oa2Kt/The John Dunbar Theme (From Dances With Wolves)/John Barry - #The John Dunbar Theme (From ##Dances With Wolves##)#.mp3
5819. FILE PATH: 
 ../data/mp3s/0H2lXaZa1VttQY33RNE1Tt/Rester la même - Edit/Lorie - Rester la même - Edit.mp3
5820. FILE PATH: 
 ../data/mp3s/0xVU3b2ZWl8cq8QuZ6oYPl/French Navy/Camera Obscura - French Navy.mp3
5821. FILE PATH: 
 ../data/mp3s/2yXyz4NLTZx9CLdXfLTp5E/I Will Not Bow/Breaking Benjamin - I Will Not Bow.mp3
5822. FILE PATH: 
 ../data/mp3s/1NnUbdEvJigY0ARnoOgDrv/Dissident - Remastered/Pearl Jam - Dissident - Remastered.mp3
5823. FILE PATH: 
 ../data/mp3s/4w3aG0qWqCQKhvsQYzX3nY/Goldberg Variations, BWV 988 Variation 7 a 1 ovvero 2 Clav./Johann Sebastian Bach, Glenn Gould - Goldberg Variations, BWV 988# Variation 7 a 1 ovvero 2 Clav..mp3
5824. FILE PATH: 
 ../data/mp3s/4T6p9mFEP9J1HbQWfdIP6F/Guilty P

/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


6683. FILE PATH: 
 ../data/mp3s/0xMd5bcWTbyXS7wPrBtZA6/Burn/Ellie Goulding - Burn.mp3
6684. FILE PATH: 
 ../data/mp3s/3blJWKRuT9BPPSi1xN8hHX/Mi Bloncito/Ñejo - Mi Bloncito.mp3
6685. FILE PATH: 
 ../data/mp3s/1eGAZX0TKa5qCxWsvhf8Th/Brick Wall Views/The Lawrence Arms - Brick Wall Views.mp3
6686. FILE PATH: 
 ../data/mp3s/7CTTTbnqMg0ASqjH8v2D9l/Things You Don't Wanna Hear/George Carlin - Things You Don't Wanna Hear.mp3
6687. FILE PATH: 
 ../data/mp3s/7025POEXvQLugcpy6cNVSq/Si Tu Quiere (feat. Pusho)/Yampi, Ozuna, Anuel AA, Pusho - Si Tu Quiere (feat. Pusho).mp3
6688. FILE PATH: 
 ../data/mp3s/3b71h4RqnMZ4d7UmTOwA0m/Constellation/Win and Woo - Constellation.mp3
6689. FILE PATH: 
 ../data/mp3s/334xEybuC25WjWcf93dzPH/Aapan Khoriya Bahaar Pha/Chorus - Aapan Khoriya Bahaar Pha.mp3
6690. FILE PATH: 
 ../data/mp3s/6JXPYFNQKDD9PipDrzxRnP/Eldorado 1997/Manu Chao - Eldorado 1997.mp3
6691. FILE PATH: 
 ../data/mp3s/4dXcDVX3HVQrHFyGE11wue/Last Chance/Anoushka Shankar - Last Chance.mp3
6692. FILE PATH

/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


6975. FILE PATH: 
 ../data/mp3s/6uv0GnArZDs61ivyZQL7kS/Then (Piano Mix)/Brad Paisley - Then (Piano Mix).mp3
6976. FILE PATH: 
 ../data/mp3s/3stWWPN41byqp8loPdy92u/Master of None/Beach House - Master of None.mp3
6977. FILE PATH: 
 ../data/mp3s/0dd2y9eFu1qkAnATCRW0G3/I Gave It All/Aquilo - I Gave It All.mp3
6978. FILE PATH: 
 ../data/mp3s/03KisebN9LuPAchPJm32M8/Feeling You/Harrison Storm - Feeling You.mp3
6979. FILE PATH: 
 ../data/mp3s/2DbCvK0YXyy6o1KLeDDDGr/Goodbye Spot - From The Good Dinosaur Score/Mychael Danna, Jeff Danna - #Goodbye Spot - From ##The Good Dinosaur## Score#.mp3
6980. FILE PATH: 
 ../data/mp3s/3TQxkQsyJS03EUwl02Qaub/Soulfight/The Revivalists - Soulfight.mp3
6981. FILE PATH: 
 ../data/mp3s/0g37m5ch9d9r9sL67qLEZ4/Lonely Night/Tribal Seeds - Lonely Night.mp3
6982. FILE PATH: 
 ../data/mp3s/5yNw7KhDPNO8g3Wfm7Eamj/Goodbye Agony/Black Veil Brides - Goodbye Agony.mp3
6983. FILE PATH: 
 ../data/mp3s/1hRDHWWealh2Pk3fnpIe75/Love Rollercoaster/Ohio Players - Love Rollercoaster.

...

10165. FILE PATH: 
 ../data/mp3s/2a7z0RvsjwSMWEPwIWO5z2/La Ultima Vez/Feid - La Ultima Vez.mp3
10166. FILE PATH: 
 ../data/mp3s/7os1J5P6JqIxKSxq71QuQC/Hero (feat. Christina Perri) - Deep Mix/Cash Cash, Christina Perri - Hero (feat. Christina Perri) - Deep Mix.mp3
10167. FILE PATH: 
 ../data/mp3s/4qZwVr553XcQNdeqjueeE3/Roses Are Red - Original Version/Aqua - Roses Are Red - Original Version.mp3
10168. FILE PATH: 
 ../data/mp3s/48DPyFzSUNGAYzXM9Zj4Jd/La rondine Chi il bel sogno di Doretta/Giacomo Puccini, Luba Orgonasova, Symfonický orchester Slovenského rozhlasu, Will Humburg - La rondine# Chi il bel sogno di Doretta.mp3
10169. FILE PATH: 
 ../data/mp3s/4xLRNLT9MkHFNpbxuvHmMC/Divertimento No. 10 in F Major, K. 247 III. Menuetto - Trio/Wolfgang Amadeus Mozart, Camerata Salzburg, Sándor Végh - Divertimento No. 10 in F Major, K. 247# III. Menuetto - Trio.mp3
10170. FILE PATH: 
 ../data/mp3s/7pUqeYHnZI8RdhGS2BkAN5/Steel On Steel/Johan Söderqvist, Patrik Andrén - Steel On Steel.mp3
10171. FI

/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/librosa/core/pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


10205. FILE PATH: 
 ../data/mp3s/3rMyMv8EjKXoPnaRo2hdJN/Stuntin' Like My Daddy - Street/Birdman, Lil Wayne - Stuntin' Like My Daddy - Street.mp3
10206. FILE PATH: 
 ../data/mp3s/3UUx0JWEJCbKZvhzpruLfe/Letting You Go/Sara Evans - Letting You Go.mp3
10207. FILE PATH: 
 ../data/mp3s/3As1HTUDBTiTDbwnuICy4C/Fisherman's Horizon Piano Collections Version (From Final Fantasy VIII) [For Piano Solo]/Nobuo Uematsu, daigoro789 - #Fisherman's Horizon# Piano Collections Version (From ##Final Fantasy VIII##) [For Piano Solo]#.mp3
10208. FILE PATH: 
 ../data/mp3s/0VSqZ3KStsjcfERGdcWpFO/Les bisous des bisounours/Le Club des Juniors - Les bisous des bisounours.mp3
10209. FILE PATH: 
 ../data/mp3s/7d1ZzOIyACB64aEibmWfH7/Release and Be Free/Kelley Hunt - Release and Be Free.mp3
10210. FILE PATH: 
 ../data/mp3s/4Zhnwb8jvP09aIRAzQMG08/Hospital Beds/Cold War Kids - Hospital Beds.mp3
10211. FILE PATH: 
 ../data/mp3s/2jx8PSP6ncYxZSKqVIoAYa/Cello Concerto in A minor, op. 129 I. Allegro - Nicht zu Schnell/Robert

/Users/vii/anaconda3/envs/brainstation_capstone/lib/python3.8/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=818
  warnings.warn(


11451. FILE PATH: 
 ../data/mp3s/2v4kQsvlTPEYOIyJkytzGH/In A Sweater Poorly Knit/mewithoutYou - In A Sweater Poorly Knit.mp3
11452. FILE PATH: 
 ../data/mp3s/1eupQ3yeBQPmAUcHyhrXkb/Mal ô mains/Sanseverino - Mal ô mains.mp3
11453. FILE PATH: 
 ../data/mp3s/6FlgXdr8JJLfQvMoHQjE0r/Hey, Snow White/Destroyer - Hey, Snow White.mp3
11454. FILE PATH: 
 ../data/mp3s/7JVlshKyXmnTVJb2H1ybFf/シューティングスター/Hachioji P, Kagamine Rin, Kagamine Len - シューティングスター.mp3
11455. FILE PATH: 
 ../data/mp3s/5JPgWqbVPYZT2k2Gjznl7N/Tyrant (feat. Daniel Caesar) - Remix/Kali Uchis, Daniel Caesar - Tyrant (feat. Daniel Caesar) - Remix.mp3
11456. FILE PATH: 
 ../data/mp3s/4khRdk0vpLBWzGu41Fd7Rl/Quisiera Alejarme (feat. Ozuna & CNCO) - Remix/Wisin, Ozuna, CNCO - Quisiera Alejarme (feat. Ozuna & CNCO) - Remix.mp3
11457. FILE PATH: 
 ../data/mp3s/5yej7ZOcsmAgfOlBsov1ED/I'll Meet You There/Phil Vassar - I'll Meet You There.mp3
11458. FILE PATH: 
 ../data/mp3s/7lAqDooEqIvJoPr6SpC8BX/It's a Fact/The Vandals - It's a Fact.mp3
1

In [14]:
vectorized_data_path = Path("../data/vectorized_mp3s/pairwise_parquets/")
path_glob = vectorized_data_path.rglob("*.parquet")
dataframes = []
for file_path in path_glob:
    tmp = pd.read_parquet(f"{file_path}")
    dataframes.append(tmp)
vectorized_df = pd.concat(dataframes)

In [16]:
output_file_date = datetime.now().strftime(
    "%Y%m%d"
)  # time stamps the file with the reverse of the date
vectorized_df.to_parquet(
    f"../data/vectorized_mp3s/pairwise_complete_parquets/{output_file_date}_complete_vectorized_data.parquet"
)

In [19]:
# data_dir = Path("../data/mp3s/")
# output_dir = Path("../data/vectorized_mp3s/pairwise_parquets/")
output_dir = Path("../data/mp3_spectra/cnn_npys/")
path_glob = output_dir.rglob("*.npy")
file_paths = []
for file_path in path_glob:
    file_paths.append(file_path)  # creates a list for repeated iteration
    # if this is not done, the .rglob command above has to be repeated to regenerate iterator
len(file_paths)

11578